
---
## |----------------------------------------- **FAST 2025** -----------------------------------|
# | **ADDED RESISTANCE & VERTICAL ACCELERATIONS FOR PLANING HULLS** |
---
|------------------------**By: Plaza, Paredes, Begovic and Datla** ------------------------|
---

## **Codes for input dataset and training code**

In [24]:
#******************************************************************************#
#                            GENERAL REQUIREMENTS                              #
#******************************************************************************#

# -------->                  General requirements                    <-------- #

import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import ScalarFormatter
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
import seaborn as sns
import random

# ---------------------------------------------------------------------------- #

def scalar(x):

    if hasattr(x, "item"):
        try:
            return x.item()
        except:
            return float(np.squeeze(x))
    else:
        return float(x)

# -------->                    GPR requirements                      <-------- #

from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C, WhiteKernel

from sklearn.gaussian_process import GaussianProcessRegressor

# -------->                    SVR requirements                      <-------- #

from sklearn.svm import SVR

# ---------------------------------------------------------------------------- #

# -------->                    ETR requirements                      <-------- #

from sklearn.ensemble import ExtraTreesRegressor

#==============================================================================#

In [25]:
#******************************************************************************#
#                                LOADING DATA                                  #
#******************************************************************************#

# -------->               Import train and test data                 <-------- #


Data_ship1 = pd.read_csv('Acc_ncg.csv')                                         # Import the train data

Data_ship2 = pd.read_csv('Acc_nBow.csv')                                        # Import the train data

Data_ship3 = pd.read_csv('AddRe.csv')                                           # Import the train data

# ---------------------------------------------------------------------------- #

# -------->                    Features data                         <-------- #
Variables1 = ['Beta [degrees]', 'Cv [-]', 'LCG [%L]', 'Tao [degrees]',
         'Fn [-]', 'H1/3 /b [-]']

X1 = Data_ship1[Variables1]                                                     # Selection of features data for ncg

Variables2 = ['Beta [degrees]',  'Cv [-]','LCG [%L]', 'Tao [degrees]',
              'H1/3 /b [-]',  'ncg [g]']
X2 = Data_ship2[Variables2]                                                     # Selection of features data for nbow

Variables3 = ['Beta [degrees]',  'Cv [-]', 'Tao [degrees]','Fn [-]',
              'H1/3 /b [-]']
X3 = Data_ship3[Variables3]                                                     # Selection of features data for AddedRes

# ---------------------------------------------------------------------------- #

# -------->                     Target data                          <-------- #

Y1 = pd.DataFrame({'ncg [g]': Data_ship1['ncg [g]']}) # target data
Y2 = pd.DataFrame({'nbow [g]': Data_ship2['nbow [g]']}) # target data
Y3 = pd.DataFrame({'Ra/wb3 [-]': Data_ship3['Ra/wb3 [-]']}) # target data
#==============================================================================#

# -------->                     Scaling data                         <-------- #

scaler_X1 = MinMaxScaler()
normalized_X1 = scaler_X1.fit_transform(X1)

scaler_X2 = MinMaxScaler()
normalized_X2 = scaler_X2.fit_transform(X2)

scaler_X3 = MinMaxScaler()
normalized_X3 = scaler_X3.fit_transform(X3)

# ---------------------------------------------------------------------------- #

# -------->            Define seed for reproducibility               <-------- #

seed_experiment = 31
np.random.seed(seed_experiment)

# ---------------------------------------------------------------------------- #

# -------->            Split bt test and training data               <-------- #

X_train1, X_test1, Y_train1, Y_test1 = train_test_split(normalized_X1,
                                                    Y1,
                                                    test_size=0.2,
                                                    random_state=42)            # random state for reproducibility

X_train2, X_test2, Y_train2, Y_test2 = train_test_split(normalized_X2,
                                                    Y2,
                                                    test_size=0.2,
                                                    random_state=42)            # random state for reproducibility

X_train3, X_test3, Y_train3, Y_test3 = train_test_split(normalized_X3,
                                                    Y3,
                                                    test_size=0.2,
                                                    random_state=42)            # random state for reproducibility

# ---------------------------------------------------------------------------- #

Y_test1 = Y_test1.to_numpy()

Y_train1 = Y_train1.to_numpy()

Y_test2 = Y_test2.to_numpy()

Y_train2 = Y_train2.to_numpy()

Y_test3 = 100*Y_test3.to_numpy()

Y_train3 = 100*Y_train3.to_numpy()

#==============================================================================#

In [26]:
X1

,Beta [degrees],Cv [-],LCG [%L],Tao [degrees],Fn [-],H1/3 /b [-]
0,10,0.6,38.0,4.0,0.594962,0.667
1,10,0.6,38.0,4.0,0.594962,0.444
2,10,0.6,40.7,4.0,1.189923,0.222
3,10,0.6,40.7,4.0,1.189923,0.444
4,10,0.6,40.7,4.0,1.189923,0.667
...,...,...,...,...,...,...
246,30,0.8,34.1,2.0,0.594962,0.351
247,30,0.8,35.3,2.0,0.892442,0.236
248,30,0.8,35.3,2.0,0.892442,0.420
249,30,0.8,35.3,2.0,0.892442,0.634


In [27]:
#==============================================================================#
# --------                  SUPPORT VECTOR MACHINE                  ---------- #
#==============================================================================#

svm_AddRe =  SVR(kernel='rbf', gamma='scale', C=10, epsilon=0.05)               # Initialize the model for added resistance

# -------->                Training AddRe SVR model                  <-------- #

svm_AddRe.fit(X_train3, Y_train3[:, 0])                                         # Fit the model with the train data and added resistance data

#==============================================================================#

#==============================================================================#
# --------               GAUSSIAN PROCESS REGRESSION                ---------- #
#==============================================================================#

# -------->                     Kernel relation                      <-------- #

kernel = C(1.0, (1e-2, 1e4)) * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 100)) + WhiteKernel(noise_level=1e-5, noise_level_bounds=(1e-8, 1.0))

# ---------------------------------------------------------------------------- #

gpr_CG =  GaussianProcessRegressor(kernel=kernel,                               # Initialize the model for nCG data
                                             n_restarts_optimizer=15)
gpr_Bow =  GaussianProcessRegressor(kernel=kernel,                              # Initialize the model for nBow data
                                             n_restarts_optimizer=15)

# -------->                 Training nCG GPR model                   <-------- #

gpr_CG.fit(X_train1, Y_train1[:, 0])                                            # Fit the model with the train data and nCG data

gpr_Bow.fit(X_train2, Y_train2[:, 0])                                           # Fit the model with the train data and nBow data

#==============================================================================#

#==============================================================================#
# --------                   EXTRA TREES REGRESSOR                  ---------- #
#==============================================================================#

etr_Bow = ExtraTreesRegressor(n_estimators=50,random_state=42)                 # Initialize the model for nBow data

etr_CG =  ExtraTreesRegressor(n_estimators=50,random_state=42)                 # Initialize the model for nCG data

etr_AddRe = ExtraTreesRegressor(n_estimators=50, random_state=42)              # Initialize the model for added resistance data

# -------->                Training nBow ETR model                   <-------- #

etr_Bow.fit(X_train2, Y_train2[:, 0])                                           # Fit the model with the train data and nBow data

etr_CG.fit(X_train1, Y_train1[:, 0])                                            # Fit the model with the train data and nBow data

etr_AddRe.fit(X_train3, Y_train3[:, 0])                                         # Fit the model with the train data and added resistance  data
#==============================================================================#

ExtraTreesRegressor(n_estimators=50, random_state=42)

## **Prediction**

In [28]:
#==============================================================================#
# --------    VERTICAL ACCELERATION AND ADDED RESISTANSE EXAMPLE    ---------- #
# ---------                   HIGH SPEED CRAFT                     ----------- #
#==============================================================================#

#                         INPUT SHIP CHARACTERISTICS                           #
#******************************************************************************#

LtBt   = 5.0                                                                    # [-] Length Beam Ratio

Beta   = 10.0                                                                   # [°] Deadrise angle

C_dis  = 0.6                                                                    # [-] Displacement Coefficient

LCG    = 38.0                                                                   # [%L] Longitudinal center of gravity

Trimt  = 4.0                                                                    # [°] Trim angle

Fnt    = 0.59                                                                   # [-] Froude number

H13    = 0.444                                                                  # [-] Normalized wave height


#******************************************************************************#
#                                 PREDICTION                                   #
#******************************************************************************#

# -------->           Input features calulation for nCG              <-------- #

X_testcg = ([ Beta, C_dis, LCG,Trimt, Fnt,  H13])                               # Data input matrix

X_testFcg = np.array(X_testcg).reshape(1, -1)                                   # X_test final matrix

X_testnomCG = scaler_X1.transform(X_testFcg)                                    # Input scaled data

# ---------------------------------------------------------------------------- #

# -------->                  nCG Prediction                          <-------- #

Ycg_gpr_e1 = gpr_CG.predict(X_testnomCG)                                        # Prediction of nCG by GPR

Ycg_etr_e1 = etr_CG.predict(X_testnomCG)                                        # Prediction of nCG by ETR

# ---------------------------------------------------------------------------- #

# -------->           Input features calulation for nBOW             <-------- #

X_testbow1 = ([ Beta, C_dis, LCG,Trimt, H13,  float(Ycg_gpr_e1)])               # Data input matrix

X_testFbow1 = np.array(X_testbow1).reshape(1, -1)                               # X_test final matrix

X_testnomBOW1 = scaler_X2.transform(X_testFbow1)                                # Input scaled data

X_testbow2 = ([ Beta, C_dis, LCG,Trimt, H13,   float(Ycg_gpr_e1)])              # Data input matrix

X_testFbow2 = np.array(X_testbow2).reshape(1, -1)                               # X_test final matrix

X_testnomBOW2 = scaler_X2.transform(X_testFbow2)                                # Input scaled data

# -------->                  nBOW Prediction                         <-------- #

Ybow_gpr_e1 = gpr_Bow.predict(X_testnomBOW1)                                      # Prediction of nBOW by GPR

Ybow_etr_e1 = etr_Bow.predict(X_testnomBOW2)                                      # Prediction of nBOW by ETR

# ---------------------------------------------------------------------------- #

# -------->          Input features calulation for Addre             <-------- #

X_testadd = ([ Beta, C_dis, Trimt, Fnt,  H13])                                  # Data input matrix

X_testFadd = np.array(X_testadd).reshape(1, -1)                                 # X_test final matrix

X_testnomADD = scaler_X3.transform(X_testFadd)                                  # Input scaled data

# -------->                  Addre Prediction                         <-------- #

Yadd_svm_e1 = 0.01*svm_AddRe.predict(X_testnomADD)                              # Prediction of Addre by SVR

Yadd_etr_e1 = 0.01*etr_AddRe.predict(X_testnomADD)                              # Prediction of Addre by ETR

#==============================================================================#

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/tmp/ipython-input-527871946.py:48: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_testbow1 = ([ Beta, C_dis, LCG,Trimt, H13,  float(Ycg_gpr_e1)])               # Data input matrix
/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(
/tmp/ipython-input-527871946.py:54: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  X_test

In [29]:
#******************************************************************************#
#                             PREDICTION RESULTS                               #
#******************************************************************************#

print('-----------------------------------------------------------------------')
print('------->  Vertical Accelerations and Addded Resistance Results <-------')
print('-----------------------------------------------------------------------')
print('')
print('________           Average vertical CG acceleration            ________')
print(f'GPR METHOD   [g]: {scalar(Ycg_gpr_e1):.4f}')
print(f'ETR METHOD   [g]: {scalar(Ycg_etr_e1):.4f}\n')

print('________          Average vertical BOW acceleration            ________')
print(f'GPR METHOD   [g]: {scalar(Ybow_gpr_e1):.4f}')
print(f'ETR METHOD   [g]: {scalar(Ybow_etr_e1):.4f}\n')

print('________             Added resistance calculation              ________')
print(f'SVR METHOD   [-]: {scalar(Yadd_svm_e1):.4f}')
print(f'ETR METHOD   [-]: {scalar(Yadd_etr_e1):.4f}\n')

print('-----------------------------------------------------------------------')

-----------------------------------------------------------------------
------->  Vertical Accelerations and Addded Resistance Results <-------
-----------------------------------------------------------------------

________           Average vertical CG acceleration            ________
GPR METHOD   [g]: 0.4253
ETR METHOD   [g]: 0.3300

________          Average vertical BOW acceleration            ________
GPR METHOD   [g]: 1.7395
ETR METHOD   [g]: 1.6411

________             Added resistance calculation              ________
SVR METHOD   [-]: 0.0144
ETR METHOD   [-]: 0.0140

-----------------------------------------------------------------------
